In [ ]:
from google.colab import files
from google.colab import drive
#GPU사용
cuda = torch.device('cuda')

#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#package install
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#KoBERT load
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup



     |████████████████████████████████| 46.9 MB 42 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=bc813c5923cc3ec6402e8b06af3cfdbca367d7b7609c5ec250573be8513f5f04
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
     |████████████████████████████████| 769 kB 4.0 MB/s 
     |████████████████████████████████| 895 kB 7.8 MB/s 
     |████████████████████████████████| 3.0 MB 18.1 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-x8ou6ble
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-

In [ ]:
!pip install sentencepiece

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()



using cached model
using cached model


In [ ]:
# PATH = '/content/drive/MyDrive/news/'


In [ ]:
#전처리
import pandas as pd
import csv
train= pd.read_csv('/content/train_data.csv')
test = pd.read_csv('/content/test_data.csv')




In [ ]:
#전처리
import re


# 형태소분석
!pip3 install konlpy
from konlpy.tag import Okt
okt=Okt()

def morph(s):
  description=''
  tmp = okt.pos(s, stem=True)
  for token,pos in tmp: 
    if pos=='Noun':
        description+=token+' '
  return description




def clean_text(sent):
  sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
  sent_clean = morph(sent_clean)
  return sent_clean

train["cleaned_title"] = train["title"].apply(lambda x : clean_text(x))
test["cleaned_title"]  = test["title"].apply(lambda x : clean_text(x))
train_text = train["cleaned_title"].tolist()
test_text = test["cleaned_title"].tolist()



In [ ]:
train_data_list = []

for q, label in zip(train['cleaned_title'], train['topic_idx'])  :
    data = []
    data.append(q)
    data.append(str(label))

    train_data_list.append(data)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 44
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
data_train = BERTDataset(train_data_list, 0, 1, tok, max_len, True, False)


using cached model


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    #     token_ids = token_ids.long().to(device)
    #     segment_ids = segment_ids.long().to(device)
    #     valid_length= valid_length
    #     label = label.long().to(device)
    #     out = model(token_ids, valid_length, segment_ids)
    #     test_acc += calc_accuracy(out, label)
    # print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.099367618560791 train acc 0.015625
epoch 1 batch id 201 loss 0.8397890329360962 train acc 0.48616293532338306
epoch 1 batch id 401 loss 0.7696942090988159 train acc 0.6425732543640897
epoch 1 batch id 601 loss 0.5087723135948181 train acc 0.7264455074875208

epoch 1 train acc 0.7439401897122486


epoch 2 batch id 1 loss 0.5516029000282288 train acc 0.875
epoch 2 batch id 201 loss 0.740095853805542 train acc 0.8348103233830846
epoch 2 batch id 401 loss 0.48506999015808105 train acc 0.8395028054862843
epoch 2 batch id 601 loss 0.4465944170951843 train acc 0.8708922628951747

epoch 2 train acc 0.8737526260504201


epoch 3 batch id 1 loss 0.36012929677963257 train acc 0.890625
epoch 3 batch id 201 loss 0.6286852955818176 train acc 0.8753109452736318
epoch 3 batch id 401 loss 0.21028587222099304 train acc 0.8822864713216958
epoch 3 batch id 601 loss 0.2906407117843628 train acc 0.9080958818635607

epoch 3 train acc 0.9115239845938375


epoch 4 batch id 1 loss 0.31887778639793396 train acc 0.9375
epoch 4 batch id 201 loss 0.4013700485229492 train acc 0.914101368159204
epoch 4 batch id 401 loss 0.13925877213478088 train acc 0.9201995012468828
epoch 4 batch id 601 loss 0.17122069001197815 train acc 0.9385659317803661

epoch 4 train acc 0.9412858893557423


epoch 5 batch id 1 loss 0.27430060505867004 train acc 0.9375
epoch 5 batch id 201 loss 0.265094518661499 train acc 0.9368781094527363
epoch 5 batch id 401 loss 0.09910435229539871 train acc 0.9422927057356608
epoch 5 batch id 601 loss 0.14357908070087433 train acc 0.9555168469217971

epoch 5 train acc 0.9563419117647058


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)



dataset_another=[]

for predict_sentence in test['cleaned_title']:
    data = [str(predict_sentence), '0']
    dataset_another.append(data)

another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

using cached model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
test_result=[]
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        
        for i in out:
          logits=i
          logits = logits.detach().cpu().numpy()
          test_result.append(np.argmax(logits))
    

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
